## Parse Data

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def list_to_plate(lst: pd.Series) -> pd.DataFrame:
    new_index = pd.MultiIndex.from_tuples([(i[0], int(i[1:])) for i in lst.index])
    plate =  pd.Series(lst.values, index=new_index).unstack()
    return plate

def plate_to_list(plate: pd.DataFrame) -> pd.Series:
    s = plate.stack()
    s.index = [f'{row}{col}' for row, col in s.index]
    return s

def set_plate_axes(df: pd.DataFrame) -> pd.DataFrame:
    """ Given a dataframe representing a multiwell plate, set axes to be proper plate axes. """

    df.columns = range(1, len(df.columns)+1)
    df.index = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'][:len(df.index)]
    return df

def bars(df, **kwargs):
    ax = sns.barplot(data=df,alpha=0.8, **kwargs)
    ax = sns.stripplot(data=df, ax=ax, size=6, dodge=True, legend=False,  edgecolor='black', linewidth=1, **kwargs)
    if 'hue' in kwargs.keys():
      plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    return ax


In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
DRIVE_FOLDER = Path('/content/drive/Shared drives/Student Biolab Zurich/iDEC/iDEC 2024/Lab results/Phage Propagation Assays/20240910_logic_validation/')


cts = pd.read_csv(DRIVE_FOLDER / '20240910_iDEC_propagation_assay_whole data.txt', sep='\t', header=1)[['Pos', 'Cp']]
layout_plate1 = pd.read_excel(DRIVE_FOLDER / 'layout_plate_1.xlsx', header=None)
layout_plate2 = pd.read_excel(DRIVE_FOLDER / 'layout_plate_2.xlsx', header=None)




MessageError: Error: credential propagation was unsuccessful

In [ ]:
cts = cts.rename(columns={'Pos': 'well', 'Cp': 'cp'})
cts = cts.set_index('well').cp
cts = list_to_plate(cts)
cts


In [ ]:
cts_plate1 = plate_to_list(set_plate_axes(cts.iloc[::2, :12])).reset_index().rename(columns={0: 'cp', 'index': 'well'})
cts_plate1['plate'] = 'plate1'
cts_plate2 = plate_to_list(set_plate_axes(cts.iloc[::2, 12:24])).reset_index().rename(columns={0: 'cp', 'index': 'well'})
cts_plate2['plate'] = 'plate2'

cts_all = pd.concat([cts_plate1, cts_plate2])
cts_all

In [ ]:
def parse_plate(df, num_cols):
  pass

def parse_layout(layout: pd.DataFrame):
  idx = 0
  plates = []
  while idx < len(layout.index)-9:
    if layout.iloc[idx, 0] is not None:
      plate = set_plate_axes(layout.iloc[idx+2:idx+2+8, 1:1+12])
      print(plate)
      plate = plate_to_list(plate)
      plate.name = layout.iloc[idx,  0]
      plates.append(plate)
      idx += 11
    else:
      idx += 1
  return pd.concat(plates, axis=1)

meta_plate1 = parse_layout(layout_plate1).reset_index().rename(columns={'index': 'well'})
meta_plate1['plate'] = 'plate1'

meta_plate2 = parse_layout(layout_plate2).reset_index().rename(columns={'index': 'well'})
meta_plate2['plate'] = 'plate2'

meta = pd.concat([meta_plate1, meta_plate2]) # this only works becase meta columns are the same, which won't be the case all the time
meta

In [ ]:
df = pd.merge(cts_all, meta, on=['well', 'plate'])
df

,well,cp,plate,strain,Phage,promoter
0,A1,20.19,plate1,1076-37-DP6,SIAH1,uninduced
1,A2,18.86,plate1,1076-37-DP6,SIAH1,uninduced
2,A3,20.47,plate1,1076-37-DP6,SIAH1,uninduced
3,A5,22.71,plate1,1076-37-DP6,SIAH2,uninduced
4,A6,22.78,plate1,1076-37-DP6,SIAH2,uninduced
...,...,...,...,...,...,...
117,E7,20.18,plate2,1113-37-DP6,SIAH2,induced
118,E9,21.30,plate2,1113-37-DP6,TadA,induced
119,E10,22.80,plate2,1113-37-DP6,TadA,induced
120,E11,22.71,plate2,1113-37-DP6,TadA,induced


## qPCR Replicate correlation

In [ ]:
replicate_correlation = df.groupby(['plate', 'qpcr replicate']).cp.std()
bad_replicates = replicate_correlation[replicate_correlation >= 2].reset_index()
df.merge(bad_replicates, on=['plate', 'qpcr replicate'], how='right')


KeyError: 'qpcr replicate'

## Compute Propagation

In [ ]:
df

,well,cp,plate,strain,Phage,promoter
0,A1,20.19,plate1,1076-37-DP6,SIAH1,uninduced
1,A2,18.86,plate1,1076-37-DP6,SIAH1,uninduced
2,A3,20.47,plate1,1076-37-DP6,SIAH1,uninduced
3,A5,22.71,plate1,1076-37-DP6,SIAH2,uninduced
4,A6,22.78,plate1,1076-37-DP6,SIAH2,uninduced
...,...,...,...,...,...,...
117,E7,20.18,plate2,1113-37-DP6,SIAH2,induced
118,E9,21.30,plate2,1113-37-DP6,TadA,induced
119,E10,22.80,plate2,1113-37-DP6,TadA,induced
120,E11,22.71,plate2,1113-37-DP6,TadA,induced


In [ ]:
import seaborn as sns

_ = bars(df[df.strain == 'LB'], x='phage', y='cp')

In [ ]:
_ = bars(df[df.strain == 'S2208'], x='phage', y='cp')

In [ ]:
df['cp_init'] = pd.NA
for phage in df.phage.unique():
  df.loc[df.phage == phage, 'cp_init'] = df[(df.phage == phage) & (df.strain == 'LB')].cp.mean()

df['fc_log2'] = df.cp_init - df.cp
df['fc'] = 2 ** df.fc_log2
df

## Results


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

_ = bars(df[(~df.substrate.isna()) & (df.substrate != 'asyn')], x='phage', y='fc_log2', hue='substrate')

In [ ]:
_ = bars(df[(~df.substrate.isna()) & (df.substrate != 'asyn')], x='phage', y='fc', hue='substrate')

## Promoters

In [ ]:
ax = bars(df[(~df.promoter.isna()) & (df.level2 == 2009)], x='phage', y='fc_log2', hue='promoter')
ax.set_title('a-syn tE1')

In [ ]:
ax = bars(df[(~df.promoter.isna()) & (df.level2 == 2008) & (df.substrate == 'asyn')], x='phage', y='fc_log2', hue='promoter')
ax.set_title('a-syn hE1')

In [ ]:
df[(~df.promoter.isna()) & (df.level2 == 2008) & (df.substrate == 'asyn') ]

In [1]:
ax = bars(df[(~df.promoter.isna()) & (df.level2 == 2008)], x='phage', y='fc_log2', hue='level')
ax.set_title('a-syn hE1')

NameError: name 'bars' is not defined

In [ ]:
ax = bars(df[(df.promoter =='weak') & (df.level2 == 2008)], x='phage', y='fc_log2', hue='strain')
ax.set_title('a-syn hE1 weak promoter')

In [ ]:
ax = bars(df[(df.promoter == 'weak') & (df.level2 == 2008)], x='phage', y='fc_log2', hue='substrate')
ax.set_title('weak promoter hE1')

In [ ]:
ax = bars(df[(df.promoter == 'strong') & (df.level2 == 2008)], x='phage', y='fc_log2', hue='substrate')
ax.set_title('weak promoter hE1')

In [ ]:
ax = bars(df[(df.promoter == 'strong')], x='phage', y='fc_log2', hue='level2')
ax.set_title('strong promoter a-syn')

In [ ]:
df[(df.promoter == 'strong')]